#
必要最小数のデータ量で、

* 分類
* 回帰
* 時系列

をautopilotで行う

https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-reference.html


CreateAutoMLJob

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_auto_ml_job.html


A minimum of 500 rows is required for the training dataset.


https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_auto_ml_job_v2.html


Creates an Amazon SageMaker AutoML job that uses non-tabular data such as images or text for Computer Vision or Natural Language Processing problems.




boto3の場合

SageMaker SDKの場合
https://sagemaker.readthedocs.io/en/stable/api/training/automl.html#sagemaker.automl.automl.AutoMLJob


https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_auto_ml_job.html

In [ ]:
'''
response = client.create_auto_ml_job(
    AutoMLJobName='string',
    InputDataConfig=[
        {
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'ManifestFile'|'S3Prefix'|'AugmentedManifestFile',
                    'S3Uri': 'string'
                }
            },
            'CompressionType': 'None'|'Gzip',
            'TargetAttributeName': 'string',
            'ContentType': 'string',
            'ChannelType': 'training'|'validation',
            'SampleWeightAttributeName': 'string'
        },
    ],
    OutputDataConfig={
        'KmsKeyId': 'string',
        'S3OutputPath': 'string'
    },
    ProblemType='BinaryClassification'|'MulticlassClassification'|'Regression',
    AutoMLJobObjective={
        'MetricName': 'Accuracy'|'MSE'|'F1'|'F1macro'|'AUC'|'RMSE'|'MAE'|'R2'|'BalancedAccuracy'|'Precision'|'PrecisionMacro'|'Recall'|'RecallMacro'
    },
    AutoMLJobConfig={
        'CompletionCriteria': {
            'MaxCandidates': 123,
            'MaxRuntimePerTrainingJobInSeconds': 123,
            'MaxAutoMLJobRuntimeInSeconds': 123
        },
        'SecurityConfig': {
            'VolumeKmsKeyId': 'string',
            'EnableInterContainerTrafficEncryption': True|False,
            'VpcConfig': {
                'SecurityGroupIds': [
                    'string',
                ],
                'Subnets': [
                    'string',
                ]
            }
        },
        'DataSplitConfig': {
            'ValidationFraction': ...
        },
        'CandidateGenerationConfig': {
            'FeatureSpecificationS3Uri': 'string',
            'AlgorithmsConfig': [
                {
                    'AutoMLAlgorithms': [
                        'xgboost'|'linear-learner'|'mlp'|'lightgbm'|'catboost'|'randomforest'|'extra-trees'|'nn-torch'|'fastai',
                    ]
                },
            ]
        },
        'Mode': 'AUTO'|'ENSEMBLING'|'HYPERPARAMETER_TUNING'
    },
    RoleArn='string',
    GenerateCandidateDefinitionsOnly=True|False,
    Tags=[
        {
            'Key': 'string',
            'Value': 'string'
        },
    ],
    ModelDeployConfig={
        'AutoGenerateEndpointName': True|False,
        'EndpointName': 'string'
    }
)
'''

## データ作成

In [ ]:
import pandas as pd

# targetカラムの値を格納するリストを作成
target_values = list(range(1, 501))
target_values100 = list(range(1, 101))

# featureカラムの値を格納するリストを作成
feature_values = target_values
feature_values100 = target_values100

# データフレームを作成
df = pd.DataFrame({'target': target_values, 'feature': feature_values})
df100 = pd.DataFrame({'target': target_values100, 'feature': feature_values100})

In [ ]:
# 確認
print(df.shape)
# データフレームの最初の5行を表示
print(df.head())

In [ ]:
df.to_csv('df_tgt_ft_500.csv', index=False, header=True)
df100.to_csv('df_tgt_ft_100.csv', index=False, header=True)

## S3に配置

In [ ]:
import boto3
import sagemaker

# SageMakerセッションを作成
sagemaker_session = sagemaker.Session()

# デフォルトのS3バケット名を取得
bucket_name = sagemaker_session.default_bucket()

# バケットが存在しない場合は作成
if not boto3.resource('s3').Bucket(bucket_name).creation_date:
    sagemaker_session.create_bucket(bucket_name)

In [ ]:
# inputフォルダを作成するためのS3クライアントを作成
s3_client = boto3.client('s3')
# inputフォルダを作成
s3_client.put_object(Bucket=bucket_name, Key='input/')
s3_client.put_object(Bucket=bucket_name, Key='output/')

In [ ]:
import os
import boto3
import sagemaker

# SageMakerセッションを作成
sagemaker_session = sagemaker.Session()

# デフォルトのS3バケット名を取得
bucket_name = sagemaker_session.default_bucket()

# inputフォルダにアップロードするファイルのパス
local_file_path = 'df_tgt_ft_500.csv'

# アップロードするファイルのS3キー
s3_key = os.path.join('input', local_file_path)

# S3クライアントを作成
s3_client = boto3.client('s3')

# ファイルをS3バケットにアップロード
s3_client.upload_file(local_file_path, bucket_name, s3_key)

print(f"{local_file_path} をデフォルトのS3バケット {bucket_name} の {s3_key} にアップロードしました。")
print(f"s3://{bucket_name}/{s3_key}")

In [ ]:
# inputフォルダにアップロードするファイルのパス
local_file_path100 = 'df_tgt_ft_100.csv'

# アップロードするファイルのS3キー
s3_key100 = os.path.join('input', local_file_path100)


# ファイルをS3バケットにアップロード
s3_client.upload_file(local_file_path100, bucket_name, s3_key)

## AutoMLジョブの発行:boto3

In [ ]:
import boto3
import sagemaker

# SageMakerセッションを作成
sagemaker_session = sagemaker.Session()

# IAMロール名を取得
role = sagemaker.get_execution_role()

print(f"現在のノートブックインスタンスのIAMロール: {role}")

In [ ]:
import boto3

client = boto3.client('sagemaker')

以下、未完成

In [ ]:
response = client.create_auto_ml_job(
    AutoMLJobName='demo1',
    InputDataConfig=[
        {
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'ManifestFile',
                    'S3Uri': 's3://sagemaker-us-east-1-805433377179/input/df_tgt_ft_500.csv'
                }
            },
            'TargetAttributeName': 'target',
        },
    ],
    OutputDataConfig={
        'S3OutputPath': 's3://sagemaker-us-east-1-805433377179/output/'
    },
    ProblemType='Regression',
    AutoMLJobObjective={
        'MetricName': 'MSE'
    },
    AutoMLJobConfig={
        'CompletionCriteria': {
            'MaxCandidates': 15,
        },
        'DataSplitConfig': {
            'ValidationFraction': 0.2
        },
        #'CandidateGenerationConfig': {
        #    'AlgorithmsConfig': [
        #        {
        #            'AutoMLAlgorithms': [
        #                'xgboost',
        #            ]
        #        },
        #    ]
        #},
        'Mode': 'AUTO'
    },
    RoleArn=role,
)

# 結果の確認

# SageMaker SDKの AutoMLで実行する場合

https://github.com/aws/amazon-sagemaker-examples/blob/main/autopilot/autopilot_customer_churn_high_level_with_evaluation.ipynb

In [ ]:
session = sagemaker.Session()

In [ ]:
from sagemaker import AutoML

In [ ]:
print(role)
print(session)

In [ ]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
base_job_name = "exp-autopilot-" + timestamp_suffix

base_job_name

AutoML()の引数は以下を参考
https://sagemaker.readthedocs.io/en/stable/api/training/automl.html#sagemaker.automl.automl.AutoML

In [ ]:
automl = AutoML(
    role=role,
    target_attribute_name='target',
    base_job_name=base_job_name,
    problem_type='Regression',
    job_objective={"MetricName":"MSE"},
    validation_fraction=0.5,
    sagemaker_session=session,
    max_candidates=20,
)

In [ ]:
train_file = './df_tgt_ft_500.csv'

In [ ]:
print(train_file)

ローカルのファイル指定でもOK

In [ ]:
automl.fit(train_file, job_name=base_job_name, wait=False, logs=True)

### 100行のデータの場合

In [ ]:
base_job_name100 = 'exp-autopilot100_1'

In [ ]:
automl = AutoML(
    role=role,
    target_attribute_name='target',
    base_job_name=base_job_name100,
    problem_type='Regression',
    job_objective={"MetricName":"MSE"},
    sagemaker_session=session,
    max_candidates=20,
)

In [ ]:
train_file100 = './df_tgt_ft_100.csv'

In [ ]:
automl.fit(train_file100, job_name=base_job_name, wait=False, logs=True)

ClientError: An error occurred (ValidationException) when calling the CreateAutoMLJob operation: Dataset is not large enough: expected minimum number of rows is 500 but only 100 were found.

必要最低行数は500だった

# 処理の流れ
* Processing Job : db
* Processing Job : pr
* Training Job : dpp0 - 4 の、　5こほど
* FeatureEngineering : Studioコンソールではくるくる処理しているのが見えるが、マネコンからは確認できず？？
* HPO : 20(max_candidates パラメータで指定） の Training Job が発行される。

https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/algorithm_mode/train.py

をみると

        else:
            num_cv_round = train_cfg.pop("_num_cv_round", 1)

            logging.info(
                "Run {}-round of {}-fold cross validation with {} rows".format(
                    num_cv_round, kfold, train_val_dmatrix.num_row()
                )
            )

## CloudWatchのログをみてみると

* HPOジョブ結果のベスト1から、該当Training Job詳細いいき、 View logsをみる

# 分割方法

https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html

ここにあるが、k-fold=5で、各20%ずつのはず。。。だが、96データがvalidation...


そして、roundが３なのは、おそらく分割方法が3パターンあるということ（seedのようなもの）

で、系15セットのavearegeで評価している（と思われる）




# 確認

* 分割はランダム?
* roundは分割の際のseedという認識でOK？

* なぜvalid data数が96なの？20%なら100では？

* 最終のモデルはアンサンブル？？（デプロイして、予測してみればわかる）

* 誤差10000くらいあるのは、MLP by mxnet: 分割サンプル数は、CloudWatchLogみてもなぞ
* 10-20くらいは、XGBoostのdpp4
* 8000くらいは、dpp2のll(Linear Learner): 50epochなので、LLってneural netなのか？
